In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import sys
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import io

# Load all

In [33]:
dpath = "/scratch/indikar_root/indikar1/cstansbu/scpc_test/align_table/"

condition = 'raw'

df = []

for f in os.listdir(dpath):


    if not condition in f:
        continue
    
    fpath = f"{dpath}{f}"
    tmp = pd.read_parquet(fpath)
    sample = f.replace(".csv", "")[0:2]
    cell = f.replace(".csv", "")[2:5]
    reference = f.split(".")[1]

    tmp['is_primary'] = tmp['align_type'] == 'primary'
    tmp['is_mapq>20'] = tmp['mapping_quality'] > 20
    tmp['is_mapq>50'] = tmp['mapping_quality'] > 50
    tmp['alignments_per_read'] = tmp.groupby('read_name')['fragment_index'].transform('count')

    summary_record = {
        'sample' : sample,
        'cell' : cell,
        'reference' : reference,
        'condition' : condition,
        'total_reads' : tmp['read_name'].nunique(),
        'total_alignments' : len(tmp),
        'alignments_per_read' : tmp['alignments_per_read'].mean(),
        'primary_alignments' : tmp['is_primary'].sum(),
        'alignments_mapq>20' : tmp['is_mapq>20'].sum(),
        'alignments_mapq>50' : tmp['is_mapq>50'].sum(),
        'mean_align_base_quality' : tmp['mean_align_base_quality'].mean(),
        'mean_mapping_quality' : tmp['mapping_quality'].mean(),
    }

    df.append(summary_record)

df = pd.DataFrame(df)
print(f"{df.shape=}")
df

df.shape=(852, 12)


,sample,cell,reference,condition,total_reads,total_alignments,alignments_per_read,primary_alignments,alignments_mapq>20,alignments_mapq>50,mean_align_base_quality,mean_mapping_quality
0,o1,b07,GRCm39,raw,126291,435946,7.731586,126291,197599,163505,15.642003,25.866018
1,o1,b66,129S1_SvImJ,raw,262786,1746697,9.271580,262786,976755,811448,12.596894,32.017957
2,o3,b79,CAST_EiJ,raw,1317519,1352547,1.253495,1317519,22401,18675,22.058849,0.951428
3,o3,b28,GRCm39,raw,68139,457908,9.073517,68139,258799,222983,12.621543,32.612988
4,o2,b96,CAST_EiJ,raw,40293,133317,8.289528,40293,62639,51716,16.716482,26.814255
...,...,...,...,...,...,...,...,...,...,...,...,...
847,o3,b12,CAST_EiJ,raw,35377,203232,8.721215,35377,112334,90817,13.666013,31.367383
848,o1,b67,129S1_SvImJ,raw,404908,2361118,8.358533,404908,1379951,1159945,13.284065,33.569954
849,o3,b24,CAST_EiJ,raw,105809,757729,10.511540,105809,449080,384210,13.459042,34.235330
850,o1,b62,CAST_EiJ,raw,268075,1681167,9.153156,268075,939106,774152,12.799751,31.906747


In [34]:
outpath = "/scratch/indikar_root/indikar1/cstansbu/scpc_test/analysis/excel_reports/alignment_summary.xlsx"
with pd.ExcelWriter(outpath, engine='xlsxwriter') as writer:

    # save each sample to a new tab
    for sample_names, group in df.groupby(['sample', 'reference']):
        sample, ref = sample_names
        group.to_excel(writer, sheet_name=f"Sample_{sample}_Ref_{ref}", index=True)

print(f'saved: {outpath}')

saved: /scratch/indikar_root/indikar1/cstansbu/scpc_test/analysis/excel_reports/alignment_summary.xlsx


In [3]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
tmp.agg(
    total_reads = ('read_name', 'nunique'),
    total_alignments = ('fragment_index', 'count'),
).head()

In [ ]:
    tmp['is_mapq_ge_20'] = tmp['mapping_quality'] > 20
    tmp['is_mapq_ge_50'] = tmp['mapping_quality'] > 50

    tmp = tmp.groupby('read_name').agg(
        total_alignments = ('fragment_index', 'count'),
        n_map_ge_20 = ('is_mapq_ge_20', 'sum'),
        n_map_ge_50 = ('is_mapq_ge_50', 'sum'),
        mean_align_base_quality = ('mean_align_base_quality', 'mean'),
        median_align_base_quality = ('mean_align_base_quality', 'median'),
        mean_mapq = ('mapping_quality', 'mean'),
        median_mapq = ('mapping_quality', 'median'),
        mean_alignment_score = ('alignment_score', 'mean'),
        median_alignment_score = ('alignment_score', 'median'),
        n_chroms = ('chrom', 'nunique'),
    ).reset_index()

    tmp['is_higher_order_candidate'] = tmp['n_map_ge_20'] > 2

In [ ]:
break